# Create Heat Map for significant Cell Cycle genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [22]:
prot_enr.res2d.head(15)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Processing of Capped Intron-Containing Pre-mRN...,123/193,3.047452e-59,4.662602e-56,0,0,4.717289,635.599053,NUP107;NUP188;EIF4A3;HNRNPU;EFTUD2;SNRPD2;SNRP...,Reactome_2016
1,Gene Expression Homo sapiens R-HSA-74160,452/1631,6.421603e-57,4.912526e-54,0,0,2.051300,265.412919,TDRKH;RPL4;ATF2;MDC1;NUP107;HNRNPU;EHMT1;PHAX;...,Reactome_2016
2,mRNA Splicing - Major Pathway Homo sapiens R-H...,92/134,8.907630e-49,4.542892e-46,0,0,5.081918,562.262229,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
3,mRNA Splicing Homo sapiens R-HSA-72172,92/144,1.076002e-44,4.115706e-42,0,0,4.729007,478.767028,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
4,Infectious disease Homo sapiens R-HSA-5663205,124/348,5.502170e-26,1.683664e-23,0,0,2.637467,153.400560,RPL4;NUP107;NUP188;RPL10A;RPL9;PSMD8;RPS15;PSM...,Reactome_2016
5,Transport of Mature Transcript to Cytoplasm Ho...,49/74,1.844730e-25,4.704062e-23,0,0,4.901274,279.138818,NUP107;NUP188;EIF4A3;MAGOH;NUP214;NDC1;NCBP1;N...,Reactome_2016
6,Major pathway of rRNA processing in the nucleo...,77/166,6.793384e-25,1.484840e-22,0,0,3.433423,191.065441,LTV1;RPL4;DDX47;RPP30;WDR3;RPLP0;PWP2;RPL10A;R...,Reactome_2016
7,rRNA processing Homo sapiens R-HSA-72312,80/180,2.691962e-24,5.148376e-22,0,0,3.289744,178.540249,LTV1;RPL4;DDX47;RPP30;WDR3;RPLP0;NAT10;PWP2;RP...,Reactome_2016
8,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",146/462,2.718785e-24,4.621935e-22,0,0,2.339136,126.925880,ANKLE2;NUP107;NUP188;ZWILCH;BUB1B;PPP2R2A;SMC4...,Reactome_2016
9,Transport of Mature mRNA derived from an Intro...,45/68,1.782404e-23,2.727077e-21,0,0,4.898332,256.581971,NUP205;NUP107;SEH1L;NUP188;RBM8A;EIF4A3;NUP160...,Reactome_2016


# Step 2: Get the list of significant genes 

In [23]:
index = 10
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

Cell Cycle Homo sapiens R-HSA-1640170
total genes: 165


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [24]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

165

In [26]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 2000)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [7]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [8]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['USP39', 'HNRNPD', 'NUP37', 'YBX1', 'NUP153', 'DDX39A', 'CD2BP2',
       'HNRNPA1', 'GTF2F1', 'HNRNPU', 'ALYREF', 'HNRNPA3', 'NUP210'],
      dtype=object)

In [9]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [10]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [11]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,101/1631,1.775164e-24,2.716001e-21,0,0,3.013392,164.796815,EHMT1;EPRS;PWP2;EFTUD2;PSMD9;SNRPD2;KHSRP;TXNL...,Reactome_2016
1,Processing of Capped Intron-Containing Pre-mRN...,36/193,3.337592e-24,2.553258e-21,0,0,9.076813,490.663374,DDX5;DHX9;DDX23;USP39;PRPF8;ELAVL1;NUP160;EFTU...,Reactome_2016
2,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",47/462,1.082883e-19,5.522702e-17,0,0,4.950443,216.183318,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;SMC4;...,Reactome_2016
3,Cell Cycle Homo sapiens R-HSA-1640170,48/566,7.248901e-17,2.772705e-14,0,0,4.126796,153.364511,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;NCAPG;BUB1B;SMC4;...,Reactome_2016
4,mRNA Splicing - Major Pathway Homo sapiens R-H...,24/134,4.208022e-16,1.287655e-13,0,0,8.715546,308.568418,DDX5;CPSF1;NCBP1;DHX9;NCBP2;CPSF3;CPSF2;DDX23;...,Reactome_2016


In [12]:
index2 = 3
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

Cell Cycle Homo sapiens R-HSA-1640170
total genes: 48


In [13]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['DCTN1', 'ACTR1A', 'MCM6', 'MCM4', 'MAD2L1', 'PCNA', 'RANGAP1',
       'RFC5', 'MCM5', 'MCM2', 'PCM1', 'MCM7', 'TPR', 'MCM3', 'RFC3',
       'SMC4', 'CDK11B', 'SMC2', 'XPO1', 'NUP160', 'PRKCB', 'NCAPD2',
       'RFC2', 'NUF2', 'RAB8A', 'RFC4', 'TOPBP1', 'NCAPG', 'RANBP2',
       'CENPF', 'NUP155', 'GINS4', 'NDC80', 'RFC1', 'OPTN', 'NCAPH',
       'PPP2R2D', 'TPX2', 'NUP85', 'RAE1', 'HAUS7', 'GINS2', 'BUB1B',
       'TOP2A', 'PSMD9', 'NUP133', 'AHCTF1', 'NUP210'], dtype=object)

In [29]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=1000)

Loading BokehJS ...

In [15]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'PCM1', 'MCM7', 'MCM3',
       'SMC4', 'CDK11B', 'SMC2', 'NCAPD2', 'RFC2', 'NUF2', 'RFC4',
       'TOPBP1', 'NCAPG', 'CENPF', 'GINS4', 'NDC80', 'TPX2', 'GINS2',
       'BUB1B', 'TOP2A', 'NUP210'], dtype=object)

In [20]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=800)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [21]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'PCM1', 'MCM7', 'MCM3', 'SMC4', 'CDK11B', 'SMC2', 'NCAPD2', 'RFC2', 'NUF2', 'RFC4', 'TOPBP1', 'NCAPG', 'CENPF', 'GINS4', 'NDC80', 'TPX2', 'GINS2', 'BUB1B', 'TOP2A', 'NUP210']


In [18]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,DNA replication Homo sapiens hsa03030,9/36,1.338080e-19,3.920575e-17,0,0,200.000000,8691.576162,RFC5;RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM6;MCM2,KEGG_2016
1,Cell cycle Homo sapiens hsa04110,7/124,1.300097e-10,1.904642e-08,0,0,45.161290,1028.025072,MCM7;MCM3;BUB1B;MCM4;MCM5;MCM6;MCM2,KEGG_2016
2,Mismatch repair Homo sapiens hsa03430,3/23,3.005178e-06,2.935058e-04,0,0,104.347826,1326.800727,RFC5;RFC4;RFC2,KEGG_2016
3,Nucleotide excision repair Homo sapiens hsa03420,3/47,2.697611e-05,1.976000e-03,0,0,51.063830,537.220023,RFC5;RFC4;RFC2,KEGG_2016
4,RNA transport Homo sapiens hsa03013,1/172,1.943127e-01,1.000000e+00,0,0,4.651163,7.619938,NUP210,KEGG_2016


In [19]:
# Look at p-values 
certain_gene = 'PRPF6'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
212,PRPF6,0.000050,0.391870,Gbm
12025,PRPF6,0.006819,0.140160,Hnscc
27881,PRPF6,0.359535,0.144750,Luad
33350,PRPF6,0.000535,0.502100,Lscc
46555,PRPF6,0.161801,0.068850,Brca
57251,PRPF6,0.338623,-0.006257,Ov
64224,PRPF6,0.000959,-0.193600,En
81255,PRPF6,0.803451,-0.063000,Colon
